# Writing out a USGSCSM ISD from a PDS3 Messenger MDIS image

In [64]:
import json
import os
import ale
from ale.drivers.kaguya_drivers import KaguyaTcPds3NaifSpiceDriver
from ale.formatters.usgscsm_formatter import to_usgscsm

## Instantiating an ALE driver

ALE drivers are objects that define how to acquire common ISD keys from an input image format, in this case we are reading in a PDS3 image using NAIF SPICE kernels for exterior orientation data. If the driver utilizes NAIF SPICE kernels, it is implemented as a [context manager](https://docs.python.org/3/reference/datamodel.html#context-managers) and will furnish metakernels when entering the context (i.e. when entering the `with` block) and free the metakernels on exit. This maintains the integrity of spicelib's internal data structures. These driver objects are short-lived and are input to a formatter function that consumes the API to create a serializable file format. `ale.formatters` contains available formatter functions. 

The default config file is located at `ale/config.yml` and is copied into your home directory at `.ale/config.yml` on first use of the library. The config file can be modified using a text editor. `ale.config` is loaded into memory as a dictionary. It is used to find metakernels for different missions. For example, there is an entry for LRO that points to `/usgs/cpkgs/isis3/data/lro/kernels/mk/` by default. If you want to use your own metakernels, you will need to udpate this path. For example, if the metakernels are located in `/data/lrolrocnac/mk/` the LRO entry should be updated with this path. If you are using the default metakernels, then you do not need to update the path.

ALE has a two step process for writing out an ISD: 1. Instantiate your driver (in this case `MessengerMdisPds3NaifSpiceDriver`) within a context and 2. pass the driver object into a formatter (in this case, `to_usgscsm`).  

Requirements:
 * A PDS3 Messenger MDIS image
 * NAIF metakernels installed
 * Config file path for LRO (ale.config.lro) pointing to LRO NAIF metakernel directory 
 * A conda environment with ALE installed into it usisng the `conda install` command or created using the environment.yml file at the base of ALE.

In [65]:
# printing config displays the yaml formatted string
print(ale.config)

# config object is a dictionary so it has the same access patterns 
print('Kaguya spice directory:', ale.config['kaguya'])

cassini: /usgs/cpkgs/isis3/data/cassini/kernels/mk/
dawn: /data/spice/dawn-m_a-spice-6-v1.0/dawnsp_1000/extras/mk
kaguya: /home/arsanders/SELENE/kernels/mk/
lro: /scratch/jlaura/spice/lro-l-spice-6-v1.0/lrosp_1000/extras/mk/
mdis: /data/spice/mess-e_v_h-spice-6-v1.0/messsp_1000/extras/mk
mro: /data/spice/mro-m-spice-6-v1.0/mrosp_1000/extras/mk
spice_root: /data/spice/

Kaguya spice directory: /home/arsanders/SELENE/kernels/mk/


In [66]:
# change to desired PDS3 image path 
fileName = '/home/arsanders/testData/TC1S2B0_01_02842S506E1942.img'

# metakernels are furnsh-ed when entering the context (with block) with a driver instance
# most driver constructors simply accept an image path 
with KaguyaTcPds3NaifSpiceDriver(fileName) as driver:
    # pass driver instance into formatter function
    usgscsmString = to_usgscsm(driver)

### Write ISD to disk 

ALE formatter functions generally return bytes or a string that can be written out to disk. ALE's USGSCSM formatter function returns a JSON encoded string that can be written out using any JSON library. 

USGSCSM requires the ISD to be colocated with the image file with a `.json` extension in place of the image extension.

In [67]:
# load the json encoded string ISD
usgscsm_dict = json.loads(usgscsmString)

# strip the image file extension and append .json 
jsonFile = os.path.splitext(fileName)[0] + '.json'

# write to disk 
with open(jsonFile, 'w') as fp:
    json.dump(usgscsm_dict, fp)

In [68]:
import knoten
import csmapi
model="USGS_ASTRO_LINE_SCANNER_SENSOR_MODEL"
plugin = csmapi.Plugin.getList()[0]
isd = csmapi.Isd(fileName)

warns = csmapi.WarningList()
if plugin.canModelBeConstructedFromISD(isd, model, warns):
    camera = plugin.constructModelFromISD(isd, model)
else:
    print("Unable to construct camera model.")
    for item in warns:
        print(item.getMessage())

In [69]:
os.environ['ISISROOT'] = '/usgs/pkgs/isis3.8.0_RC1/install'

In [70]:
from pysis import isis

In [71]:
from pysis.exceptions import ProcessError

cub_loc = '/home/arsanders/usgscsm/tests/data/TC1S2B0_01_02842S506E1942.cub'

try: 
    isis.kaguyatc2isis(from_=fileName, to=cub_loc)
except ProcessError as e:
    print(e.stderr)

In [72]:
try:
    isis.spiceinit(from_=cub_loc, shape='ellipsoid')
except ProcessError as e:
    print(e.stderr)

In [73]:
output = isis.campt(from_=cub_loc)

In [74]:
import pvl
import numpy as np
pvl_output = pvl.loads(output)

In [75]:
bodyfixed = pvl_output['GroundPoint']['BodyFixedCoordinate']
bodyfixed = np.asarray(bodyfixed.value) * 1000
bodyfixed

array([-1068288.7882488 ,  -270668.36351983, -1343151.6898293 ])

In [76]:
from knoten import csm
line = pvl_output['GroundPoint']['Line'] -.5
sample = pvl_output['GroundPoint']['Sample'] -.5
point = csm.generate_ground_point(0, (line,  sample), camera)
point.x, point.y, point.z

(-1068290.489959783, -270678.0831693551, -1343148.3776386175)

In [77]:
point = np.array([point.x, point.y, point.z])
point - bodyfixed

array([-1.70171098, -9.71964953,  3.31219068])